## Declare Models

Here, we define _module-level_ constructs that will __form the structures__ which we will be `querying` from the database. This structure, known as a `Declarative Mapping`, _defines at once_ both a __Python object model__, as well as __database metadata__ that _describes real SQL tables_ that `exist, or will exist`, in a particular database.

In [8]:
from sqlalchemy import Column, ForeignKey, Integer, String, create_engine, select
from sqlalchemy.orm import Session, declarative_base, relationship

In [2]:
Base = declarative_base()

In [3]:
class User(Base):
    __tablename__ = "user_account"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    
    addresses = relationship("Address", back_populates="user", cascade="all, delete-orphan")
    
    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

In [4]:
class Address(Base):
    __tablename__ = "address"
    
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"), nullable=False)
    
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

The _mapping_ starts with a `base class`, which above is called `Base`, and is created by calling upon the `declarative_base()` function, which produces a __new `base` class__.

_Individual mapped classes_ are then _created_ by making __subclasses of `Base`__. A _mapped class_ typically refers to a `single particular database table`, the name of which is indicated by using the __`__tablename__`__ _class-level attribute_.

Next, _columns_ that are part of the table are declared, by _adding attributes_ __linked__ to the `Column` construct. `Column` describes _all aspects_ of a `database column`, including __typing information__ with _type objects_ such as `Integer` and `String` as well as __server defaults__ and __constraint information__, such as _membership_ within the `primary key` and `foreign keys`.

_All_ `ORM mapped classes` __require__ _at least_ __one column__ be declared as part of the `primary key`, typically by using the `Column.primary_key` parameter on those `Column` objects that should be part of the key. In the above example, the `User.id` and `Address.id` columns are _marked_ as __primary key__.

Taken together, the _combination_ of a `string table name` as well as a `list of column declarations` is __referred__ towards in SQLAlchemy as __`table metadata`__. Setting up `table metadata` using both `Core` and `ORM` approaches is introduced in the `SQLAlchemy 1.4/2.0 Tutorial` at `Working with Database Metadata`. The above mapping is an example of what's referred towards as `Declarative Table configuration`.

Other _Declarative directives_ are available, most commonly the `relationship()` construct indicated above. In contrast to the _column-based attributes_, `relationship()` denotes a __linkage between two `ORM classes`__. In the above example, `User.addresses` __links `User` to `Address`__, and `Address.user` __links `Address` to `User`__. The `relationship()` construct is introduced in the `SQLAlchemy 1.4/2.0 Tutorial` at `Working with Related Objects`.

Finally, the above example classes include a `__repr__()` method, which is _not required_ but is __useful for debugging__.

## Create an Engine

The `Engine` is a _factory_ that can __create new database connections__ for us, which also _holds onto connections_ inside of a `Connection Pool` for __fast reuse__. For learning purposes, we normally use a `SQLite` __memory-only__ database for convenience.

In [5]:
engine = create_engine("sqlite://", echo=True, future=True)

> ##### Tips
> 
> The `echo=True` parameter indicates that __SQL emitted by connections__ will be _logged to standard out_. `future=True` is to __ensure__ we are using the _latest SQLAlchemy 2.0-style APIs_.

## Emit CREATE TABLE DDL

Using our `table metadata` and our `engine`, we can _generate_ our `schema` __at once__ in our __target `SQLite` database__, using a method called `MetaData.create_all()`.

In [6]:
Base.metadata.create_all(engine)

2022-10-13 11:07:52,292 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-13 11:07:52,294 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-10-13 11:07:52,296 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-13 11:07:52,298 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-10-13 11:07:52,299 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-13 11:07:52,301 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-10-13 11:07:52,303 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-13 11:07:52,305 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-10-13 11:07:52,306 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-13 11:07:52,309 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-10-13 11:07:52,311 INFO sqlalchemy.engine.Engine [no key 0.00228s] ()
2022-10-13 11:07:52,313 INFO sqlalchemy.engine.Engine 
C

## Create Objects and Persist

We are now ready to `insert` data in the database. We accomplish this by _creating instances_ of `User` and `Address` classes, which have an `__init__()` method already as __established automatically__ by the __`declarative mapping process`__. We then _pass them to the database_ using an object called a `Session`, which makes use of the `Engine` to __interact with the `database`__. The `Session.add_all()` method is used here to __add multiple objects at once__, and the `Session.commit()` method will be used to __flush any pending changes__ to the database and then __commit__ the _current database transaction_, which is __always in progress__ whenever the `Session` is used.

In [7]:
with Session(engine) as session:
    spongebob = User(
        name="spongebob",
        fullname="Spongebob Squarepants",
        addresses=[Address(email_address="spongebob@sqlalchemy.org")]
    )
    sandy = User(
        name="sandy",
        fullname="Sandy Cheeks",
        addresses=[
            Address(email_address="sandy@sqlalchemy.org"),
            Address(email_address="sandy@squirrelpower.org"),
        ]
    )
    patrick = User(name="patrick", fullname="Patrick Star")
    
    session.add_all([spongebob, sandy, patrick])
    session.commit()

2022-10-13 11:10:39,458 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-13 11:10:39,461 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-10-13 11:10:39,462 INFO sqlalchemy.engine.Engine [generated in 0.00158s] ('spongebob', 'Spongebob Squarepants')
2022-10-13 11:10:39,464 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-10-13 11:10:39,465 INFO sqlalchemy.engine.Engine [cached since 0.004276s ago] ('sandy', 'Sandy Cheeks')
2022-10-13 11:10:39,467 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-10-13 11:10:39,469 INFO sqlalchemy.engine.Engine [cached since 0.00816s ago] ('patrick', 'Patrick Star')
2022-10-13 11:10:39,472 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?)
2022-10-13 11:10:39,473 INFO sqlalchemy.engine.Engine [generated in 0.00100s] ('spongebob@sqlalchemy.org', 1)
2022-10-13 11:10:39,475 INFO sqlalchemy.

> ##### Tips
> 
> It's recommended that the `Session` be __used in context manager style__ as above, that is, using the Python `with: statement`. The `Session` object __represents__ _active database resources_ so it's good to make sure it's __closed out__ when a _series of operations_ are __`completed`__. In the next section, we'll keep a `Session` opened just for illustration purposes.

## Simple SELECT

With some rows in the database, here's the _simplest form_ of __emitting__ a `SELECT` statement _to load some objects_. To create `SELECT` statements, we use the `select()` function to __create a new `Select` object__, which we then _invoke_ using a `Session`. The method that is _often useful_ when __querying for `ORM` objects__ is the `Session.scalars()` method, which will return a `ScalarResult` object that will _iterate_ through the ORM objects we've selected.

In [9]:
session = Session(engine)

stmt = select(User).where(User.name.in_(["spongebob", "sandy"]))
for user in session.scalars(stmt):
    print(user)

2022-10-13 11:45:02,486 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-13 11:45:02,491 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name IN (?, ?)
2022-10-13 11:45:02,493 INFO sqlalchemy.engine.Engine [generated in 0.00209s] ('spongebob', 'sandy')
User(id=1, name='spongebob', fullname='Spongebob Squarepants')
User(id=2, name='sandy', fullname='Sandy Cheeks')


The above query also made use of the `Select.where()` method to __add `WHERE` criteria__, and also used the `ColumnOperators.in_()` method that's part of all SQLAlchemy _column-like constructs_ to use the SQL `IN operator`.

In [ ]:
# 1

## SELECT with JOIN

It's _very common_ to `query` amongst __multiple tables at once__, and in SQL the `JOIN` _keyword_ is the _primary way_ this happens. The `Select` construct __creates joins__ using the `Select.join()` method.

In [10]:
stmt = (
    select(Address).
    join(Address.user).
    where(User.name == "sandy").
    where(Address.email_address == "sandy@sqlalchemy.org")
)
sandy_address = session.scalars(stmt).one()
print(f"{sandy_address = }")

2022-10-13 11:49:10,880 INFO sqlalchemy.engine.Engine SELECT address.id, address.email_address, address.user_id 
FROM address JOIN user_account ON user_account.id = address.user_id 
WHERE user_account.name = ? AND address.email_address = ?
2022-10-13 11:49:10,881 INFO sqlalchemy.engine.Engine [generated in 0.00170s] ('sandy', 'sandy@sqlalchemy.org')
sandy_address = Address(id=2, email_address='sandy@sqlalchemy.org')


The above query illustrates __multiple `WHERE` criteria__ which are __automatically chained together__ using `AND`, as well as how to use SQLAlchemy _column-like objects_ to __create `"equality"` comparisons__, which uses the _overridden_ Python method `ColumnOperators.__eq__()` to produce a _SQL criteria object_.

In [ ]:
# 2

## Make Changes

The `Session` object, in conjunction with our _ORM-mapped classes_ `User` and `Address`, __automatically track changes to the objects__ as they are made, which result in _SQL statements_ that will be __emitted the next time the `Session` flushes__. Below, we _change_ one `email address` _associated_ with `"sandy"`, and also __add a new email address__ to `"patrick"`, _after emitting_ a `SELECT` to __retrieve the row for `"patrick"`__.

In [11]:
stmt = select(User).where(User.name == "patrick")
patrick = session.scalars(stmt).one()
patrick.addresses.append(Address(email_address="patrickstar@sqlalchemy.org"))
sandy_address.email_address = "sandy_cheeks@sqlalchemy.org"
session.commit()

2022-10-13 12:02:56,569 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-10-13 12:02:56,570 INFO sqlalchemy.engine.Engine [generated in 0.00161s] ('patrick',)
2022-10-13 12:02:56,575 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2022-10-13 12:02:56,578 INFO sqlalchemy.engine.Engine [generated in 0.00291s] (3,)
2022-10-13 12:02:56,584 INFO sqlalchemy.engine.Engine UPDATE address SET email_address=? WHERE address.id = ?
2022-10-13 12:02:56,585 INFO sqlalchemy.engine.Engine [generated in 0.00162s] ('sandy_cheeks@sqlalchemy.org', 2)
2022-10-13 12:02:56,587 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?)
2022-10-13 12:02:56,588 INFO sqlalchemy.engine.Engine [cached since 3137s ago] ('patrickstar@sqlalchemy.org', 3)

Notice when we accessed `patrick.addresses`, a `SELECT` was _emitted_. This is called a __`lazy load`__. Background on different ways to _access related items_ using more or less SQL is introduced at `Loader Strategies`.

In [ ]:
# 3

## Some Deletes

All things must come to an end, as is the case for some of our database rows - here's a quick demonstration of __two different forms of `deletion`__, both of which are _important based on the specific use case_.

First we will _remove_ one of the `Address` objects from the `"sandy"` user. When the `Session` _next flushes_, this will __result in the row being deleted__. This behavior is something that we _configured in our mapping_ called the __`delete cascade`__. We can get a handle to the `sandy` object by _primary key_ using `Session.get()`, then work with the object.

In [12]:
sandy = session.get(User, 2)
sandy.addresses.remove(sandy_address)

2022-10-13 12:09:04,844 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-13 12:09:04,848 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2022-10-13 12:09:04,849 INFO sqlalchemy.engine.Engine [generated in 0.00172s] (2,)
2022-10-13 12:09:04,852 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2022-10-13 12:09:04,853 INFO sqlalchemy.engine.Engine [cached since 368.3s ago] (2,)


The _last_ `SELECT` above was the __`lazy load`__ operation proceeding so that the `sandy.addresses` collection could be _loaded_, so that we could _remove_ the `sandy_address` member. There are _other ways_ to go about this _series of operations_ that _won't emit as much SQL_.

We can _choose to emit_ the `DELETE` SQL for what's _set to be changed so far_, __without committing the transaction__, using the `Session.flush()` method.

In [13]:
session.flush()

2022-10-13 12:11:02,258 INFO sqlalchemy.engine.Engine DELETE FROM address WHERE address.id = ?
2022-10-13 12:11:02,259 INFO sqlalchemy.engine.Engine [generated in 0.00158s] (2,)


Next, we will _delete_ the `"patrick"` user _entirely_. For a _top-level delete_ of an object by itself, we use the `Session.delete()` method; this method __doesn't actually perform the deletion__, but __sets up the object to be deleted__ on the _next flush_. The operation will __also cascade to related objects__ _based on the cascade_ options that we _configured_, in this case, onto the _related `Address` objects_.

In [14]:
session.delete(patrick)

2022-10-13 12:17:11,143 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2022-10-13 12:17:11,144 INFO sqlalchemy.engine.Engine [cached since 486.3s ago] (3,)
2022-10-13 12:17:11,147 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2022-10-13 12:17:11,149 INFO sqlalchemy.engine.Engine [cached since 854.6s ago] (3,)


The `Session.delete()` method in this particular case __emitted two `SELECT` statements__, even though it __didn't emit a DELETE__, which _might seem surprising_. This is because when the method went to inspect the object, it turns out the `patrick` object was __expired__, which happened when we _last called upon_ `Session.commit()`, and the SQL __emitted__ was to __re-load the rows__ from the `new transaction`. This `expiration` is _optional_, and in normal use we will often be turning it off for situations where it doesn't apply well.

To illustrate the rows being deleted, here's the commit:

In [15]:
session.commit()

2022-10-13 12:28:15,506 INFO sqlalchemy.engine.Engine DELETE FROM address WHERE address.id = ?
2022-10-13 12:28:15,507 INFO sqlalchemy.engine.Engine [cached since 1033s ago] (4,)
2022-10-13 12:28:15,511 INFO sqlalchemy.engine.Engine DELETE FROM user_account WHERE user_account.id = ?
2022-10-13 12:28:15,513 INFO sqlalchemy.engine.Engine [generated in 0.00150s] (3,)
2022-10-13 12:28:15,515 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
# 4